In [48]:
import pandas as pd
import numpy as np
import os  
#os.makedirs('work', exist_ok=True)
data_df= pd.read_csv('waster_gen.csv')
emi_df= pd.read_csv('waste_gen_carbon_emission.csv')

com_df = pd.merge(data_df, emi_df, on=['country', 'waste_type', 'waste_treatment', 'year'], how="left")
com_df.fillna({"waste_produced_in_tonnes":0,
                "water_waste_produced_in_m3":0,
                "emission_factor(kgCo2e)_pt":0,
              "emission_factor(kgCo2e)_pm3":0}, inplace=True)

com_df["carbon_emission(kgCo2e)"] = com_df.apply(lambda x: x["emission_factor(kgCo2e)_pt"]*x["waste_produced_in_tonnes"] if x["waste_type"] =="Plastic"
                                                else x["emission_factor(kgCo2e)_pm3"]*x["water_waste_produced_in_m3"], axis=1)


com_df["scope"] = "3"
com_df["calculation_method"] = "waste-types pecific method"
output_cols = ["country", "year", "waste_type", "waste_produced_in_tonnes", "water_waste_produced_in_m3", "waste_treatment","scope","calculation_method", "carbon_emission(kgCo2e)"]


#Setting up the emission to zero to test avg method

# com_df.loc[com_df['waste_treatment']=="Landfill", 'carbon_emission(kgCo2e)'] = 0
# com_df.loc[com_df['waste_treatment']=="Wastewater", 'carbon_emission(kgCo2e)'] = 0

# Average method

com_df['total_waste_tonnes'] = com_df['waste_produced_in_tonnes'].sum()
com_df['total_water_waste'] = com_df['water_waste_produced_in_m3'].sum()

com_df_avg_emission_kgCo2e_pt = com_df.groupby(by=['waste_treatment'], as_index=False).agg({'emission_factor(kgCo2e)_pt':'mean'})
com_df_avg_emission_kgCo2e_pm3 = com_df.groupby(by=['waste_treatment'], as_index=False).agg({'emission_factor(kgCo2e)_pm3':'mean'})


df_with_avg = pd.merge(com_df, com_df_avg_emission_kgCo2e_pt, on=['waste_treatment'], how='outer')
final_df_avg_m3 = pd.merge(df_with_avg, com_df_avg_emission_kgCo2e_pm3, on=['waste_treatment'], how='outer')


final_df_avg_m3['ave_emi'] = final_df_avg_m3.apply(lambda x : x['total_waste_tonnes']*(x['waste_produced_in_tonnes']/x['total_waste_tonnes'])*x['emission_factor(kgCo2e)_pt_y']
                                                                                     if (x['waste_treatment'] == "Landfill" or x['waste_treatment'] == "Incinerated with energy recovery" or x['waste_treatment'] == "Recycled")
                                                                                     else x['total_water_waste']*(x['water_waste_produced_in_m3']/x['total_water_waste'])*x['emission_factor(kgCo2e)_pm3_y'], axis=1)

final_df_avg_m3['CO2e_Calculation_Method_New'] = final_df_avg_m3.apply(lambda x : "Average Method"
                                                                                     if (x['carbon_emission(kgCo2e)']==0)
                                                                                      else x['calculation_method'], axis=1)


final_df_avg_m3['calculation_method'] = final_df_avg_m3['CO2e_Calculation_Method_New']
final_df_avg_m3['carbon_emission(kgCo2e)'] = final_df_avg_m3['ave_emi']
final_df_avg_m3['carbon_emission(kgCo2e)'] = np.round(final_df_avg_m3['carbon_emission(kgCo2e)'], decimals = 3)

final_df_avg_m3.to_csv('work/output.csv', columns=output_cols, index=False)
print("Done")



Done


In [46]:
com_df.columns

Index(['country', 'year', 'waste_type', 'waste_produced_in_tonnes',
       'water_waste_produced_in_m3', 'waste_treatment',
       'emission_factor(kgCo2e)_pt', 'emission_factor(kgCo2e)_pm3',
       'carbon_emission(kgCo2e)', 'scope', 'calculation_method',
       'total_waste_tonnes', 'total_water_waste'],
      dtype='object')